In [15]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
#  设置图片大小
rcParams["figure.figsize"]=12,4
target = "result"
gender = "gender"

In [16]:
# 读取特征预处理后的数据（石头自己私下根据原文一步一步的步骤处理特征，这里不再展开了）
# train = pd.read_csv("train3.csv",encoding="ISO-8859-1")
train = pd.read_csv("train3.csv")
# 检测特征是否有nan
train.isnull().sum()
train.columns
# 定义训练数据标记
target = "Disbursed"
# 选择特征
predictors =[x for x in train.columns if x not in [target]]

In [17]:
# 构建xgb模型并执行交叉验证，该模型可以验证更新后的模型好坏
# 定义modelfit函数，根据一定的学习率得到最优迭代次数
def modelfit(alg, dtrain, predictors, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
                          metrics='auc', early_stopping_rounds=early_stopping_rounds, verbose_eval=True)
        alg.set_params(n_estimators=cvresult.shape[0])

    # Fit the algorithm on the data
    alg.fit(dtrain[predictors].values, dtrain[target].values, eval_metric='auc')

    # Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors].values)
    dtrain_predprob = alg.predict_proba(dtrain[predictors].values)[:, 1]

    # Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain[target].values, dtrain_predprob))

    feat_imp = pd.Series(alg.get_booster().get_fscore()).sort_values(ascending=False)  #降序查看特征重要性程度

In [18]:
# 第一步：设置默认的模型常数
predictors = [x for x in train.columns if x not in [target]]  # 选择特征
xgb1 = XGBClassifier( 
 learning_rate =0.1,              # 典型值范围：0.01～0.2
 n_estimators=1000,               # 迭代次数第一次一般选择较多一点
 max_depth=5,                     # 典型值范围：3～10
 min_child_weight=1,              # 最小叶子节点最小权重和为1，让树尽可能的生长
 gamma=0,                         # 不限制节点分裂，
 subsample=0.8,                   # 每次树构建选择的样本采样比例，典型值范围：0.5~1
 colsample_bytree=0.8,            # 每次树构建时选择的特征采样比例，典型值范围：0.5~1
 objective= 'binary:logistic',    # 目标损失函数采用二分类的logistic回归函数
 nthread=1,                       # 默认是选择最大的线程数，这里选择4
 scale_pos_weight=1,              # 正负比例极其不平衡，因此设置1
 seed=27)                         # 设置seed为相同的整数，使结果具有重复性
modelfit(xgb1, train, predictors)

[0]	train-auc:0.780466+0.0139895	test-auc:0.775581+0.0354966
[1]	train-auc:0.806229+0.00406123	test-auc:0.799038+0.0205264
[2]	train-auc:0.811157+0.00617447	test-auc:0.801318+0.018375
[3]	train-auc:0.813346+0.00613748	test-auc:0.803481+0.0170953
[4]	train-auc:0.816768+0.00661876	test-auc:0.804061+0.0166806
[5]	train-auc:0.820121+0.00487614	test-auc:0.805812+0.0194123
[6]	train-auc:0.822352+0.00548285	test-auc:0.806501+0.0182012
[7]	train-auc:0.824336+0.00599396	test-auc:0.808624+0.0174738
[8]	train-auc:0.826115+0.00575724	test-auc:0.809013+0.0173059
[9]	train-auc:0.828254+0.006193	test-auc:0.810213+0.0172003
[10]	train-auc:0.83129+0.00543369	test-auc:0.813065+0.0175405
[11]	train-auc:0.833959+0.0040921	test-auc:0.815133+0.0179863
[12]	train-auc:0.835468+0.00422095	test-auc:0.81632+0.0175617
[13]	train-auc:0.837163+0.00442354	test-auc:0.816288+0.0183132
[14]	train-auc:0.838183+0.00413645	test-auc:0.816608+0.0188523
[15]	train-auc:0.839484+0.00429647	test-auc:0.816931+0.0190975
[16]	trai

[131]	train-auc:0.900811+0.0015078	test-auc:0.837512+0.012961
[132]	train-auc:0.901125+0.00143937	test-auc:0.837435+0.0128159
[133]	train-auc:0.901575+0.00140894	test-auc:0.837467+0.0127526
[134]	train-auc:0.90199+0.00153534	test-auc:0.837323+0.0128476
[135]	train-auc:0.902285+0.00163563	test-auc:0.837302+0.012717
[136]	train-auc:0.902578+0.00165176	test-auc:0.837349+0.0126642
[137]	train-auc:0.902949+0.00156486	test-auc:0.837444+0.0125177
[138]	train-auc:0.90346+0.00167513	test-auc:0.837503+0.0124501
[139]	train-auc:0.903773+0.00163717	test-auc:0.83742+0.0124095
[140]	train-auc:0.904092+0.00160947	test-auc:0.837545+0.0124288
[141]	train-auc:0.904425+0.00163261	test-auc:0.83734+0.0124351
[142]	train-auc:0.904643+0.00174796	test-auc:0.837444+0.0123483
[143]	train-auc:0.904916+0.00183084	test-auc:0.837412+0.0123772
[144]	train-auc:0.90547+0.00208184	test-auc:0.837549+0.012421
[145]	train-auc:0.905775+0.00203672	test-auc:0.837469+0.0125278
[146]	train-auc:0.905961+0.00192178	test-auc:0.83

In [23]:
# 第二步：树的最大深度和最小树叶子节点权重调优。由第一步得到最优迭代数是198，因此更新该参数为198
param_test1 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=198, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train[predictors].values,train[target].values)
gsearch1.best_params_, gsearch1.best_score_

({'max_depth': 3, 'min_child_weight': 5}, 0.831642684956385)

In [24]:
#第三步：gamma参数调优。由第二步得到最优决策树的最大深度是3，最小叶子节点权重是5，更新max_depth和min_child_weight参数
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=198, max_depth = 3,
 min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(train[predictors].values,train[target].values)
gsearch3.best_params_, gsearch3.best_score_

({'gamma': 0.0}, 0.831642684956385)

In [26]:
# 第四步：subsample and colsample_bytree参数调优。由第三步得到最优的Gamma参数是0，更新gamma参数
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=198, max_depth = 3,
 min_child_weight = 5, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(train[predictors].values,train[target].values)
gsearch4.best_params_, gsearch4.best_score_

({'colsample_bytree': 0.9, 'subsample': 0.8}, 0.8325318765589031)

In [27]:
# 第五步：正则化参数调优 。由第四步得到最优的subsample and colsample_bytree，colsample_bytree＝0.9，subsample ＝ 0.8
param_test6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=198, max_depth = 3,
 min_child_weight = 5, gamma=0, subsample=0.8, colsample_bytree=0.9,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(train[predictors].values,train[target].values)
gsearch6.best_params_, gsearch6.best_score_

({'reg_alpha': 1}, 0.8328047815496173)

In [28]:
# 第六步：学习率参数调优,减小学习率，最优迭代次数也会相应的改变，增大最大迭代数 。由第四步得到最优的reg_alpha是1
xgb4 = XGBClassifier(
 learning_rate =0.01,
 n_estimators=5000,
 max_depth=3,
 min_child_weight=5,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.9,
 reg_alpha=1,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb4, train, predictors)

[0]	train-auc:0.736331+0.0135779	test-auc:0.733516+0.014179
[1]	train-auc:0.741524+0.0116124	test-auc:0.73903+0.0176179
[2]	train-auc:0.742801+0.0117697	test-auc:0.740293+0.017985
[3]	train-auc:0.746056+0.00770903	test-auc:0.74397+0.0198942
[4]	train-auc:0.747493+0.00707626	test-auc:0.745382+0.02092
[5]	train-auc:0.748792+0.00620758	test-auc:0.746661+0.0220229
[6]	train-auc:0.749176+0.00621639	test-auc:0.746828+0.0223785
[7]	train-auc:0.749625+0.00636289	test-auc:0.747006+0.0222617
[8]	train-auc:0.750643+0.00717532	test-auc:0.747641+0.0219805
[9]	train-auc:0.750658+0.00718553	test-auc:0.747594+0.021886
[10]	train-auc:0.750609+0.00719783	test-auc:0.747683+0.0217804
[11]	train-auc:0.756749+0.0138203	test-auc:0.753562+0.0245663
[12]	train-auc:0.756814+0.0137245	test-auc:0.753802+0.0246545
[13]	train-auc:0.75848+0.0141369	test-auc:0.755008+0.0241928
[14]	train-auc:0.758487+0.0141446	test-auc:0.754979+0.0241834
[15]	train-auc:0.758575+0.0140586	test-auc:0.755171+0.0243074
[16]	train-auc:0.7

[131]	train-auc:0.812376+0.0041343	test-auc:0.806874+0.0182154
[132]	train-auc:0.812387+0.00415677	test-auc:0.806878+0.0180154
[133]	train-auc:0.812436+0.00424367	test-auc:0.806938+0.0179091
[134]	train-auc:0.812531+0.00424577	test-auc:0.807003+0.0179266
[135]	train-auc:0.812591+0.00431396	test-auc:0.806986+0.0177649
[136]	train-auc:0.812647+0.00428988	test-auc:0.807097+0.0178592
[137]	train-auc:0.812703+0.00425743	test-auc:0.807141+0.0178995
[138]	train-auc:0.812703+0.00426412	test-auc:0.807187+0.017917
[139]	train-auc:0.812723+0.0042629	test-auc:0.807182+0.0179425
[140]	train-auc:0.81283+0.00427158	test-auc:0.80721+0.0179918
[141]	train-auc:0.81292+0.00419714	test-auc:0.807243+0.0180072
[142]	train-auc:0.812936+0.00419467	test-auc:0.807219+0.0180076
[143]	train-auc:0.812997+0.00422885	test-auc:0.807248+0.0179019
[144]	train-auc:0.813026+0.00420675	test-auc:0.807264+0.0179541
[145]	train-auc:0.813125+0.00427047	test-auc:0.807587+0.0179535
[146]	train-auc:0.813141+0.00427928	test-auc:0

[260]	train-auc:0.818942+0.00400173	test-auc:0.811493+0.0169857
[261]	train-auc:0.818974+0.0040187	test-auc:0.811456+0.0169606
[262]	train-auc:0.819036+0.00402806	test-auc:0.81153+0.0169746
[263]	train-auc:0.819061+0.00405215	test-auc:0.811567+0.0170116
[264]	train-auc:0.819097+0.00401748	test-auc:0.81163+0.0170027
[265]	train-auc:0.819164+0.00405048	test-auc:0.811636+0.0170334
[266]	train-auc:0.819218+0.0040606	test-auc:0.8117+0.0170513
[267]	train-auc:0.819252+0.00409204	test-auc:0.811774+0.0170499
[268]	train-auc:0.819399+0.00416734	test-auc:0.811776+0.0170712
[269]	train-auc:0.819442+0.00418963	test-auc:0.811872+0.0170024
[270]	train-auc:0.819532+0.00420645	test-auc:0.811952+0.0169876
[271]	train-auc:0.819586+0.00422201	test-auc:0.811946+0.0169504
[272]	train-auc:0.819647+0.00420126	test-auc:0.811995+0.0169471
[273]	train-auc:0.819657+0.00419006	test-auc:0.81197+0.0168984
[274]	train-auc:0.819673+0.00417945	test-auc:0.811949+0.0169237
[275]	train-auc:0.819778+0.00420158	test-auc:0.

[390]	train-auc:0.824767+0.00382343	test-auc:0.81534+0.0174756
[391]	train-auc:0.824787+0.00382481	test-auc:0.815294+0.0175076
[392]	train-auc:0.824843+0.0038188	test-auc:0.815334+0.0174737
[393]	train-auc:0.824876+0.0038076	test-auc:0.815382+0.0174115
[394]	train-auc:0.824928+0.00379488	test-auc:0.815415+0.0174179
[395]	train-auc:0.824963+0.00379552	test-auc:0.815439+0.0174431
[396]	train-auc:0.825015+0.00380282	test-auc:0.815484+0.0174419
[397]	train-auc:0.82505+0.00379835	test-auc:0.815494+0.0174222
[398]	train-auc:0.825071+0.00383323	test-auc:0.815556+0.0173863
[399]	train-auc:0.825099+0.00384777	test-auc:0.815542+0.0173853
[400]	train-auc:0.825138+0.00383562	test-auc:0.815529+0.0173711
[401]	train-auc:0.825169+0.00384297	test-auc:0.815541+0.0173927
[402]	train-auc:0.825205+0.00381968	test-auc:0.815566+0.0174146
[403]	train-auc:0.825229+0.00381361	test-auc:0.815617+0.0174602
[404]	train-auc:0.825298+0.00380955	test-auc:0.815636+0.0174607
[405]	train-auc:0.825346+0.00378178	test-auc

[520]	train-auc:0.829964+0.00346873	test-auc:0.819598+0.0174553
[521]	train-auc:0.830012+0.00346852	test-auc:0.819627+0.0174601
[522]	train-auc:0.830056+0.00346046	test-auc:0.81969+0.0174475
[523]	train-auc:0.8301+0.00345519	test-auc:0.819725+0.0174508
[524]	train-auc:0.830131+0.00344213	test-auc:0.819741+0.0174761
[525]	train-auc:0.830172+0.0034389	test-auc:0.819776+0.0174721
[526]	train-auc:0.830225+0.00343632	test-auc:0.819824+0.0174576
[527]	train-auc:0.830295+0.00343243	test-auc:0.819855+0.0174403
[528]	train-auc:0.830357+0.00342169	test-auc:0.819918+0.0174633
[529]	train-auc:0.830395+0.00339605	test-auc:0.819951+0.0174876
[530]	train-auc:0.830447+0.00339098	test-auc:0.81998+0.0175167
[531]	train-auc:0.830464+0.00338684	test-auc:0.82003+0.0174995
[532]	train-auc:0.830504+0.00336953	test-auc:0.820078+0.017493
[533]	train-auc:0.830541+0.00335245	test-auc:0.820066+0.0174979
[534]	train-auc:0.830597+0.00333788	test-auc:0.8201+0.0175198
[535]	train-auc:0.83064+0.00333314	test-auc:0.820

...

[1798]	train-auc:0.86547+0.00275225	test-auc:0.837238+0.0142747
[1799]	train-auc:0.865491+0.00274527	test-auc:0.837244+0.0142664
[1800]	train-auc:0.865506+0.00273648	test-auc:0.837251+0.0142669
[1801]	train-auc:0.865529+0.0027392	test-auc:0.837255+0.0142613
[1802]	train-auc:0.865546+0.00274016	test-auc:0.837264+0.0142541
[1803]	train-auc:0.865567+0.0027404	test-auc:0.837279+0.014247
[1804]	train-auc:0.865578+0.00274262	test-auc:0.837274+0.01425
[1805]	train-auc:0.865604+0.00274894	test-auc:0.837286+0.01424
[1806]	train-auc:0.865616+0.00274726	test-auc:0.837308+0.0142386
[1807]	train-auc:0.865637+0.00275286	test-auc:0.837308+0.0142365
[1808]	train-auc:0.865647+0.00275199	test-auc:0.837302+0.0142441
[1809]	train-auc:0.865662+0.00275215	test-auc:0.837296+0.0142445
[1810]	train-auc:0.865672+0.00275007	test-auc:0.837304+0.0142342
[1811]	train-auc:0.865693+0.0027487	test-auc:0.837313+0.0142277
[1812]	train-auc:0.865706+0.00274041	test-auc:0.837309+0.0142219
[1813]	train-auc:0.865715+0.002739

[1925]	train-auc:0.867479+0.00267683	test-auc:0.837632+0.0140279
[1926]	train-auc:0.867493+0.0026676	test-auc:0.837634+0.0140245
[1927]	train-auc:0.867509+0.00267167	test-auc:0.837636+0.0140342
[1928]	train-auc:0.867526+0.00267895	test-auc:0.837635+0.0140428
[1929]	train-auc:0.867544+0.00267291	test-auc:0.83764+0.0140319
[1930]	train-auc:0.867563+0.0026694	test-auc:0.837645+0.0140319
[1931]	train-auc:0.867577+0.00266413	test-auc:0.837636+0.0140315
[1932]	train-auc:0.867585+0.00266098	test-auc:0.837643+0.0140174
[1933]	train-auc:0.867601+0.0026511	test-auc:0.837649+0.0140054
[1934]	train-auc:0.867613+0.00264598	test-auc:0.837654+0.0140062
[1935]	train-auc:0.867626+0.00265088	test-auc:0.83765+0.0140057
[1936]	train-auc:0.867639+0.00264902	test-auc:0.837653+0.0140086
[1937]	train-auc:0.867654+0.00265921	test-auc:0.837658+0.014011
[1938]	train-auc:0.867668+0.00266082	test-auc:0.837671+0.014002
[1939]	train-auc:0.867678+0.0026571	test-auc:0.837669+0.0140048
[1940]	train-auc:0.867693+0.00265

[2052]	train-auc:0.869357+0.00265823	test-auc:0.838005+0.0137988
[2053]	train-auc:0.869367+0.00265997	test-auc:0.838015+0.0137937
[2054]	train-auc:0.869377+0.00266106	test-auc:0.838018+0.013792
[2055]	train-auc:0.869387+0.00265931	test-auc:0.83802+0.0137803
[2056]	train-auc:0.869394+0.00266268	test-auc:0.838017+0.0137755
[2057]	train-auc:0.869412+0.00267382	test-auc:0.838014+0.0137862
[2058]	train-auc:0.869428+0.00267724	test-auc:0.838019+0.0137669
[2059]	train-auc:0.869448+0.00267574	test-auc:0.838033+0.0137578
[2060]	train-auc:0.869459+0.00267729	test-auc:0.838035+0.0137545
[2061]	train-auc:0.869473+0.00267855	test-auc:0.838026+0.0137569
[2062]	train-auc:0.869488+0.00268392	test-auc:0.838041+0.0137533
[2063]	train-auc:0.869499+0.00268055	test-auc:0.838046+0.0137616
[2064]	train-auc:0.869519+0.00268112	test-auc:0.838061+0.0137455
[2065]	train-auc:0.869539+0.00268507	test-auc:0.838063+0.0137368
[2066]	train-auc:0.869561+0.00268729	test-auc:0.838057+0.0137326
[2067]	train-auc:0.869579+0

[2179]	train-auc:0.871125+0.00262854	test-auc:0.838224+0.0135328
[2180]	train-auc:0.871144+0.00262203	test-auc:0.83822+0.0135422
[2181]	train-auc:0.87115+0.00262073	test-auc:0.83823+0.0135461
[2182]	train-auc:0.871167+0.00262612	test-auc:0.838228+0.0135478
[2183]	train-auc:0.87118+0.00262643	test-auc:0.838235+0.0135496
[2184]	train-auc:0.871193+0.00261947	test-auc:0.838244+0.0135471
[2185]	train-auc:0.871204+0.00262355	test-auc:0.83825+0.0135467
[2186]	train-auc:0.871219+0.00262521	test-auc:0.838244+0.0135346
[2187]	train-auc:0.871227+0.00262344	test-auc:0.838244+0.0135391
[2188]	train-auc:0.871243+0.00262175	test-auc:0.838253+0.0135398
[2189]	train-auc:0.871257+0.00261861	test-auc:0.838255+0.0135518
[2190]	train-auc:0.871272+0.00262005	test-auc:0.838241+0.0135491
[2191]	train-auc:0.871284+0.00261678	test-auc:0.838245+0.0135503
[2192]	train-auc:0.871289+0.00261418	test-auc:0.83825+0.0135485
[2193]	train-auc:0.871297+0.0026085	test-auc:0.838249+0.0135543
[2194]	train-auc:0.871302+0.0026

[2306]	train-auc:0.872772+0.00256154	test-auc:0.838399+0.0134601
[2307]	train-auc:0.872792+0.0025607	test-auc:0.838397+0.0134494
[2308]	train-auc:0.872797+0.00255795	test-auc:0.838403+0.0134481
[2309]	train-auc:0.872812+0.00256128	test-auc:0.838409+0.0134475
[2310]	train-auc:0.87283+0.00255499	test-auc:0.838405+0.0134297
[2311]	train-auc:0.872835+0.00255839	test-auc:0.838399+0.0134287
[2312]	train-auc:0.872847+0.00255725	test-auc:0.838389+0.0134235
[2313]	train-auc:0.872854+0.00255762	test-auc:0.838385+0.013429
[2314]	train-auc:0.872871+0.00255068	test-auc:0.838393+0.0134242
[2315]	train-auc:0.872878+0.00255112	test-auc:0.838394+0.013432
[2316]	train-auc:0.87289+0.00255233	test-auc:0.838397+0.0134313
[2317]	train-auc:0.872899+0.00255181	test-auc:0.838397+0.013429
[2318]	train-auc:0.872908+0.00254966	test-auc:0.838393+0.0134313
[2319]	train-auc:0.872923+0.00254652	test-auc:0.838376+0.0134407
[2320]	train-auc:0.87294+0.00255276	test-auc:0.838372+0.0134244
[2321]	train-auc:0.872956+0.0025

In [ ]:
# 模型的交叉验证率由0.836增加到0.838，略微的增加